# 句子情感分类实验

本notebook实现了基于不同深度学习模型的句子情感分类任务，包括：
- MLP（多层感知机）
- CNN（卷积神经网络）
- RNN（LSTM/GRU）

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import wandb

from Dataset.data_processor import load_word2vec, create_data_loader
from model.mlp import MLP
from model.cnn import TextCNN
from model.rnn import RNNClassifier
from utils.trainer import Trainer

In [2]:
!wandb login 1372d7543091de94fe28f909a22670e57518697f

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


## 1. 数据准备
加载预训练的word2vec模型和数据集

In [3]:
# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 加载word2vec模型
word2vec_model = load_word2vec('Dataset/wiki_word2vec_50.bin')

# 创建数据加载器
train_loader = create_data_loader('Dataset/train.txt', word2vec_model, batch_size=32)
val_loader = create_data_loader('Dataset/validation.txt', word2vec_model, batch_size=32)
test_loader = create_data_loader('Dataset/test.txt', word2vec_model, batch_size=32)

## 2. wandb配置
设置wandb项目和实验配置函数

In [4]:
# 设置wandb项目和实验配置
def init_wandb_experiment(config, name=None):
    """初始化wandb实验"""
    run = wandb.init(
        project="sentiment-analysis",
        name=name if name else config['model_name'],
        config=config,
        reinit=True
    )
    return run

# 模型性能比较函数
def compare_models(models_metrics):
    """比较不同模型的性能并在wandb中可视化"""
    # 创建比较表格
    comparison_table = wandb.Table(
        columns=["Model", "Accuracy", "Precision", "Recall", "F1-Score"]
    )
    
    # 添加各模型的性能数据
    for model_name, metrics in models_metrics.items():
        comparison_table.add_data(
            model_name,
            metrics['test_accuracy'],
            metrics['test_precision'],
            metrics['test_recall'],
            metrics['test_f1']
        )
    
    # 记录表格
    wandb.log({"model_comparison": comparison_table})
    
    # 创建性能对比图
    for metric in ['test_accuracy', 'test_precision', 'test_recall', 'test_f1']:
        metric_data = [[model, metrics[metric]] for model, metrics in models_metrics.items()]
        table = wandb.Table(data=metric_data, columns=["model", metric])
        wandb.log({
            f"{metric}_comparison": wandb.plot.bar(
                table, "model", metric, title=f"Model Comparison - {metric}"
            )
        })

## 3. 模型训练与评估
### 3.1 MLP模型

In [5]:
# MLP配置
mlp_config = {
    'model_name': 'MLP',
    'input_dim': 50,  # word2vec维度
    'hidden_dim': 256,
    'learning_rate': 0.001,
    'num_epochs': 10
}

# 初始化wandb实验 - MLP
mlp_run = init_wandb_experiment(mlp_config)

# 初始化MLP模型
mlp_model = MLP(input_dim=mlp_config['input_dim'], hidden_dim=mlp_config['hidden_dim'])

# 记录MLP模型结构
wandb.watch(mlp_model, log="all")

# 训练MLP
mlp_trainer = Trainer(mlp_model, device, mlp_config)
mlp_trainer.train(train_loader, val_loader, mlp_config['num_epochs'])

# 测试MLP
mlp_test_metrics = mlp_trainer.test(test_loader)
print('MLP Test Results:', mlp_test_metrics)

# 记录MLP测试结果
wandb.log(mlp_test_metrics)

# 保存MLP模型到wandb
mlp_artifact = wandb.Artifact(f"mlp_model", type="model")
mlp_artifact.add_file("best_model.pth")
mlp_run.log_artifact(mlp_artifact)
mlp_run.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 3555125903 (3555125903-tsinghua-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


/root/PA2/PA2_Sentiment_Analysis/Dataset/data_processor.py:38: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  word_vectors = torch.tensor(word_vectors, dtype=torch.float32)


Epoch 1/10:
Train Loss: 0.5462, Train F1: 0.7243
Val Loss: 0.5062, Val F1: 0.7292

Epoch 2/10:
Train Loss: 0.5030, Train F1: 0.7625
Val Loss: 0.4873, Val F1: 0.7623

Epoch 3/10:
Train Loss: 0.4981, Train F1: 0.7608
Val Loss: 0.4811, Val F1: 0.7681

Epoch 4/10:
Train Loss: 0.4927, Train F1: 0.7636
Val Loss: 0.4889, Val F1: 0.7762

Epoch 5/10:
Train Loss: 0.4878, Train F1: 0.7678
Val Loss: 0.4878, Val F1: 0.7465

Epoch 6/10:
Train Loss: 0.4856, Train F1: 0.7703
Val Loss: 0.4790, Val F1: 0.7781

Epoch 7/10:
Train Loss: 0.4844, Train F1: 0.7698
Val Loss: 0.4793, Val F1: 0.7691

Epoch 8/10:
Train Loss: 0.4809, Train F1: 0.7706
Val Loss: 0.4789, Val F1: 0.7547

Epoch 9/10:
Train Loss: 0.4780, Train F1: 0.7730
Val Loss: 0.4726, Val F1: 0.7717

Epoch 10/10:
Train Loss: 0.4767, Train F1: 0.7738
Val Loss: 0.4708, Val F1: 0.7700



train_accuracy,▁▆▆▇▇█▇███
train_f1,▁▆▆▇▇▇▇███
train_loss,█▄▃▃▂▂▂▁▁▁
train_precision,▁▆▆▇▇█▇▇██
train_recall,▁▇▆▆▇▇▇███
val_accuracy,▁▅▇▂▅▅▇▆▇█
val_f1,▁▆▇█▃█▇▅▇▇
val_loss,█▄▃▅▄▃▃▃▁▁
val_precision,▇▅▆▁█▂▅█▅▆
val_recall,▁▄▅█▂▇▅▃▅▄
train_accuracy,0.77088


MLP Test Results: {'test_accuracy': 0.7723577235772358, 'test_precision': 0.8159509202453987, 'test_recall': 0.7112299465240641, 'test_f1': 0.76}


Error: You must call wandb.init() before wandb.log()

### 3.2 CNN模型

In [6]:
# CNN配置
cnn_config = {
    'model_name': 'CNN',
    'input_dim': 50,
    'num_filters': 100,
    'filter_sizes': (3, 4, 5),
    'learning_rate': 0.001,
    'num_epochs': 10
}
wandb.init()
# 初始化wandb实验 - CNN
cnn_run = init_wandb_experiment(cnn_config)

# 初始化CNN模型
cnn_model = TextCNN(input_dim=cnn_config['input_dim'],
                   num_filters=cnn_config['num_filters'],
                   filter_sizes=cnn_config['filter_sizes'])

# 记录CNN模型结构
wandb.watch(cnn_model, log="all")

# 训练CNN
cnn_trainer = Trainer(cnn_model, device, cnn_config)
cnn_trainer.train(train_loader, val_loader, cnn_config['num_epochs'])

# 测试CNN
cnn_test_metrics = cnn_trainer.test(test_loader)
print('CNN Test Results:', cnn_test_metrics)

# 记录CNN测试结果
wandb.log(cnn_test_metrics)

# 保存CNN模型到wandb
cnn_artifact = wandb.Artifact(f"cnn_model", type="model")
cnn_artifact.add_file("best_model.pth")
cnn_run.log_artifact(cnn_artifact)
cnn_run.finish()

/root/miniconda3/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/10:
Train Loss: 0.5223, Train F1: 0.7388
Val Loss: 0.4549, Val F1: 0.7769



/root/miniconda3/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 2/10:
Train Loss: 0.4411, Train F1: 0.7958
Val Loss: 0.4430, Val F1: 0.7782

Epoch 3/10:
Train Loss: 0.4014, Train F1: 0.8173
Val Loss: 0.4118, Val F1: 0.8142



/root/miniconda3/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


...

/root/miniconda3/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 6/10:
Train Loss: 0.3119, Train F1: 0.8659
Val Loss: 0.4066, Val F1: 0.8266

Epoch 7/10:
Train Loss: 0.2842, Train F1: 0.8807
Val Loss: 0.4146, Val F1: 0.8283

Epoch 8/10:
Train Loss: 0.2579, Train F1: 0.8952
Val Loss: 0.4067, Val F1: 0.8202

Epoch 9/10:
Train Loss: 0.2350, Train F1: 0.9019
Val Loss: 0.4147, Val F1: 0.8286

Epoch 10/10:
Train Loss: 0.2251, Train F1: 0.9055
Val Loss: 0.4139, Val F1: 0.8255



train_accuracy,▁▃▄▅▆▆▇███
train_f1,▁▃▄▅▆▆▇███
train_loss,█▆▅▄▄▃▂▂▁▁
train_precision,▁▄▅▅▆▆▇███
train_recall,▁▃▄▅▆▆▇▇██
val_accuracy,▁▃▅▆▇▇▇███
val_f1,▁▁▆▅███▇██
val_loss,█▇▃▂▁▂▃▂▃▃
val_precision,▃█▃█▄▂▁▇▅▆
val_recall,▂▁▆▄▇▇█▅▆▆
train_accuracy,0.90559


CNN Test Results: {'test_accuracy': 0.8482384823848238, 'test_precision': 0.8742857142857143, 'test_recall': 0.8181818181818182, 'test_f1': 0.8453038674033149}


Error: You must call wandb.init() before wandb.log()

### 3.3 RNN模型（LSTM）

In [7]:
# LSTM配置
lstm_config = {
    'model_name': 'LSTM',
    'input_dim': 50,
    'hidden_dim': 256,
    'num_layers': 2,
    'bidirectional': True,
    'learning_rate': 0.001,
    'num_epochs': 20
}
wandb.init()
# 初始化wandb实验 - LSTM
lstm_run = init_wandb_experiment(lstm_config)

# 初始化LSTM模型
lstm_model = RNNClassifier(input_dim=lstm_config['input_dim'],
                         hidden_dim=lstm_config['hidden_dim'],
                         num_layers=lstm_config['num_layers'],
                         rnn_type='LSTM',
                         bidirectional=lstm_config['bidirectional'])

# 记录LSTM模型结构
wandb.watch(lstm_model, log="all")

# 训练LSTM
lstm_trainer = Trainer(lstm_model, device, lstm_config)
lstm_trainer.train(train_loader, val_loader, lstm_config['num_epochs'])

# 测试LSTM
lstm_test_metrics = lstm_trainer.test(test_loader)
print('LSTM Test Results:', lstm_test_metrics)

# 记录LSTM测试结果
wandb.log(lstm_test_metrics)

# 保存LSTM模型到wandb
lstm_artifact = wandb.Artifact(f"lstm_model", type="model")
lstm_artifact.add_file("best_model.pth")
lstm_run.log_artifact(lstm_artifact)
lstm_run.finish()

Epoch 1/20:
Train Loss: 0.6920, Train F1: 0.5460
Val Loss: 0.6931, Val F1: 0.6642

Epoch 2/20:
Train Loss: 0.6941, Train F1: 0.5128
Val Loss: 0.6936, Val F1: 0.6638

Epoch 3/20:
Train Loss: 0.6927, Train F1: 0.5236
Val Loss: 0.6922, Val F1: 0.6647

Epoch 4/20:
Train Loss: 0.6927, Train F1: 0.5415
Val Loss: 0.6920, Val F1: 0.6677

Epoch 5/20:
Train Loss: 0.6921, Train F1: 0.5668
Val Loss: 0.6932, Val F1: 0.6648

Epoch 6/20:
Train Loss: 0.6925, Train F1: 0.5492
Val Loss: 0.6935, Val F1: 0.6639

Epoch 7/20:
Train Loss: 0.6912, Train F1: 0.5529
Val Loss: 0.6915, Val F1: 0.6691

Epoch 8/20:
Train Loss: 0.6733, Train F1: 0.6257
Val Loss: 0.6553, Val F1: 0.6918

Epoch 9/20:
Train Loss: 0.6285, Train F1: 0.6923
Val Loss: 0.5578, Val F1: 0.7203

Epoch 10/20:
Train Loss: 0.5271, Train F1: 0.7511
Val Loss: 0.4814, Val F1: 0.7658

Epoch 11/20:
Train Loss: 0.4778, Train F1: 0.7777
Val Loss: 0.4658, Val F1: 0.7692

Epoch 12/20:
Train Loss: 0.4504, Train F1: 0.7946
Val Loss: 0.4466, Val F1: 0.7934

E

train_accuracy,▁▁▁▁▁▁▁▂▄▅▆▆▇▇▇▇▇███
train_f1,▂▁▁▂▂▂▂▃▄▅▆▆▆▇▇▇▇▇██
train_loss,████████▇▅▄▄▄▃▃▃▂▂▂▁
train_precision,▁▁▁▁▁▁▁▂▃▅▆▆▆▇▇▇▇███
train_recall,▂▁▂▂▃▃▂▅▆▆▆▆▇▇▇▇▇▇██
val_accuracy,▁▁▁▁▁▁▁▄▆▇▇▇▇███████
val_f1,▁▁▁▁▁▁▁▂▃▅▆▇▇▇███▇█▇
val_loss,███████▇▅▃▂▂▂▁▁▁▁▁▂▃
val_precision,▁▁▁▁▁▁▁▃▆▇█▇▆█▇█▇█▇█
val_recall,███████▄▁▃▂▃▆▄▄▄▄▃▅▃
train_accuracy,0.88674


LSTM Test Results: {'test_accuracy': 0.8157181571815718, 'test_precision': 0.8606060606060606, 'test_recall': 0.7593582887700535, 'test_f1': 0.8068181818181818}


Error: You must call wandb.init() before wandb.log()

### 3.4 RNN模型（GRU）

In [8]:
# GRU配置
gru_config = {
    'model_name': 'GRU',
    'input_dim': 50,
    'hidden_dim': 256,
    'num_layers': 2,
    'bidirectional': True,
    'learning_rate': 0.001,
    'num_epochs': 20
}

# 初始化wandb实验 - GRU
gru_run = init_wandb_experiment(gru_config)

# 初始化GRU模型
gru_model = RNNClassifier(input_dim=gru_config['input_dim'],
                        hidden_dim=gru_config['hidden_dim'],
                        num_layers=gru_config['num_layers'],
                        rnn_type='GRU',
                        bidirectional=gru_config['bidirectional'])

# 记录GRU模型结构
wandb.watch(gru_model, log="all")

# 训练GRU
gru_trainer = Trainer(gru_model, device, gru_config)
gru_trainer.train(train_loader, val_loader, gru_config['num_epochs'])

# 测试GRU
gru_test_metrics = gru_trainer.test(test_loader)
print('GRU Test Results:', gru_test_metrics)

# 记录GRU测试结果
wandb.log(gru_test_metrics)

# 保存GRU模型到wandb
gru_artifact = wandb.Artifact(f"gru_model", type="model")
gru_artifact.add_file("best_model.pth")
gru_run.log_artifact(gru_artifact)
gru_run.finish()

Epoch 1/20:
Train Loss: 0.6578, Train F1: 0.5789
Val Loss: 0.5278, Val F1: 0.7563

Epoch 2/20:
Train Loss: 0.5024, Train F1: 0.7633
Val Loss: 0.4723, Val F1: 0.7541

Epoch 3/20:
Train Loss: 0.4482, Train F1: 0.7945
Val Loss: 0.4518, Val F1: 0.7978

Epoch 4/20:
Train Loss: 0.4202, Train F1: 0.8111
Val Loss: 0.4480, Val F1: 0.8106

Epoch 5/20:
Train Loss: 0.4011, Train F1: 0.8202
Val Loss: 0.4189, Val F1: 0.8104

Epoch 6/20:
Train Loss: 0.3768, Train F1: 0.8327
Val Loss: 0.4030, Val F1: 0.8113

Epoch 7/20:
Train Loss: 0.3455, Train F1: 0.8487
Val Loss: 0.4134, Val F1: 0.8053

Epoch 8/20:
Train Loss: 0.3161, Train F1: 0.8655
Val Loss: 0.4372, Val F1: 0.8210

Epoch 9/20:
Train Loss: 0.2734, Train F1: 0.8865
Val Loss: 0.4249, Val F1: 0.8021

Epoch 10/20:
Train Loss: 0.2243, Train F1: 0.9088
Val Loss: 0.4760, Val F1: 0.8144

Epoch 11/20:
Train Loss: 0.1701, Train F1: 0.9341
Val Loss: 0.6284, Val F1: 0.7630

Epoch 12/20:
Train Loss: 0.1312, Train F1: 0.9517
Val Loss: 0.7470, Val F1: 0.8105

E

train_accuracy,▁▄▅▅▅▅▆▆▆▇▇▇████████
train_f1,▁▄▅▅▅▅▆▆▆▇▇▇████████
train_loss,█▆▆▅▅▅▅▄▄▃▃▂▂▂▁▁▁▁▁▁
train_precision,▁▄▅▅▅▅▆▆▆▇▇▇████████
train_recall,▁▄▅▅▅▅▆▆▆▆▇▇████████
val_accuracy,▁▃▆▆▆▇▇█▇▇▅▇▇▇▇▆▆▇▆▆
val_f1,▁▁▆▇▇▇▆█▆▇▂▇▇█▇▆▆▇▇▇
val_loss,▂▂▂▁▁▁▁▁▁▂▃▅▆▅▆▆█▇▇▇
val_precision,▁▅▄▃▃▆▆▆▆▄█▆▅▄▅▅▅▃▄▄
val_recall,▅▂▆██▆▅▇▅▇▁▆▆█▇▆▆█▇▇
train_accuracy,0.99105


GRU Test Results: {'test_accuracy': 0.8238482384823849, 'test_precision': 0.8388888888888889, 'test_recall': 0.8074866310160428, 'test_f1': 0.8228882833787466}


Error: You must call wandb.init() before wandb.log()

## 4. 模型性能比较

In [9]:
# 模型性能比较
models_metrics = {
    'MLP': mlp_test_metrics,
    'CNN': cnn_test_metrics,
    'LSTM': lstm_test_metrics,
    'GRU': gru_test_metrics
}

# 初始化比较实验
comparison_run = init_wandb_experiment({"comparison": True}, name="model_comparison")

# 可视化比较结果
compare_models(models_metrics)
comparison_run.finish()